In [1]:
#pip install yfinance
#pip install panel
#pip install hvplot

In [31]:
from pathlib import Path
import pandas as pd
import yfinance as yf
import numpy as np
import panel as pn
pn.extension()
import hvplot.pandas
import seaborn as sns
%matplotlib inline

## Data pull

In [3]:
# manual inputs, to be replaced by stock and portfolio dfs and variables from analytics team

tickr = "NIO"
tickrs = ["META","AMZN","GOOG","AAPL","NFLX"]
mkttickr = "^GSPC"

#variable assignment
stock = yf.Ticker(tickr)
portfolio = yf.Tickers(tickrs)
mkt = yf.Ticker(mkttickr)
ticker_names = [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],"SP500"]

In [4]:
#historical data dataframes

stock_history_df = stock.history(period = "5y")
portfolio_history_df = portfolio.history(period = "5y")
mkt_history_df = mkt.history(period = "5y")

# #preview
# portfolio_history_df.tail()

[*********************100%***********************]  5 of 5 completed


## Df setup

In [5]:
#prices df
prices_df = pd.concat([
     stock_history_df["Close"],portfolio_history_df["Close"],mkt_history_df["Close"]],
     axis = 'columns',
     join = 'inner'
    )

#renaming columns, plotting prices (no SP500)
prices_df.columns = [ticker_names[0],ticker_names[1],ticker_names[2],ticker_names[3],ticker_names[4],ticker_names[5],ticker_names[6]]

In [6]:
# combined returns dataframe (close price, daily, 5 years)

#combine data frames using only Close column
returns_df = pd.concat([
     stock_history_df["Close"],portfolio_history_df["Close"],mkt_history_df["Close"]],
     axis = 'columns',
     join = 'inner'
)
#renaming columns
returns_df.columns = [ticker_names[0],ticker_names[1],ticker_names[2],ticker_names[3],ticker_names[4],ticker_names[5],ticker_names[6]]

#calculate returns
returns_df = returns_df.pct_change().dropna()

In [7]:
# Cummulative returns
cumulative_returns = (1 + returns_df).cumprod()

## Calculations

In [8]:
#correlation
correlation = returns_df.corr()
# correlation

In [9]:
#sharpe ratio calculation
sharpe_ratios = (returns_df.mean() * 252) / (returns_df.std() * np.sqrt(252))
sharpe_ratios = sharpe_ratios.sort_values()
sharpe_ratios = sharpe_ratios.to_frame()
sharpe_ratios.columns = ["Sharpe"]
# sharpe_ratios

In [10]:
#sortino ratios
sortino_ratios = (returns_df.mean() * 252) / (returns_df[returns_df<0].std() * np.sqrt(252))
sortino_ratios = sortino_ratios.sort_values()
sortino_ratios = sortino_ratios.to_frame()
sortino_ratios.columns = ["Sortino"]
# sortino_ratios

In [11]:
#volatility 
volatility = returns_df.std() * np.sqrt(252)
volatility = volatility.sort_values()
volatility = volatility.to_frame()
volatility.columns = ["Volatility"]

In [12]:
# Stock Covariance Calculation
covariance_df = returns_df.cov()
covariance_df = covariance_df[["SP500"]]
covariance_df.columns = ["Covariance"]
# display(covariance_df)

#beta calculation
variance = returns_df['SP500'].var()
# print(variance)

beta_df = returns_df.cov()/variance
beta_df = beta_df[["SP500"]]
beta_df.columns = ["Beta"]
# beta_df

## Plotting

### Closing Prices 

In [13]:
# ticker_price_wd = pn.widgets.CheckButtonGroup(options= list(prices_df.columns), value = [prices_df.columns[0]], name='Ticker')

# def pricesplot_df(ticker_price_wd):
#     df = pd.DataFrame(prices_df[ticker_price_wd])
#     return df

# prices_plot = hvplot.bind(pricesplot_df, ticker_price_wd).interactive(width=600)
# prices_plot.head()


In [14]:
prices_plot = prices_df.drop(columns = "SP500").hvplot.line(
    title = "Historical Prices - 5 years",
    ylabel = "Stock Price"
    )
prices_plot 

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

### Daily Returns 

In [15]:
# ireturns = returns_df.interactive()
# returns_df.hvplot.bar()

## Cummulative Returns 

In [41]:
cum_returns_plot = cumulative_returns.hvplot(
    title = "Historical Cummulative Returns - 5 years",
    ylabel = "Return %"
    )
cum_returns_plot 

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [38]:
tickers_wd = pn.widgets.Select(options= [tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr], name='Ticker')
# tickers_wd
# def returnsplot_df(tickers_wd):
#     df = pd.DataFrame(cumulative_returns[tickers_wd])
#     return df

# returns_plot = hvplot.bind(returnsplot_df,tickers_wd).interactive(width=600).hvplot(
#     title = "Historical Cummulative Returns - 5 years",
#     ylabel = "Return %"
#     )

# returns_plot

In [37]:
# def pricesplot_df(tickers_wd):
#     df = pd.DataFrame(prices_df[tickers_wd])
#     return df

# pricesplot = hvplot.bind(pricesplot_df,tickers_wd).interactive(width=600).hvplot(
#     title = "Historical Prices - 5 years",
#     ylabel = "Stock Price"
#     )

# pricesplot 

## Ratios table

In [19]:
#Create df starting by correlation
ratios_df = returns_df.corr()
ratios_df.drop(ratios_df.columns[0:-1],axis=1,inplace = True) #drop columns for correlation against each other, leave only correlation against SP500
ratios_df.rename(columns = {"SP500":"Correlation"},inplace = True) #rename column from SP500 to Correlation
ratios_df["Volatility"] = returns_df.std() * np.sqrt(252)
ratios_df["Sharpe"] =  (returns_df.mean() * 252) / (returns_df.std() * np.sqrt(252))
ratios_df["Sortino"] = (returns_df.mean() * 252) / (returns_df[returns_df<0].std() * np.sqrt(252))
ratios_df["Covariance"] = covariance_df["Covariance"]
ratios_df["Beta"] = beta_df["Beta"]
ratios_df


,Correlation,Volatility,Sharpe,Sortino,Covariance,Beta
NIO,0.299314,0.998569,0.609783,1.213404,0.000271,1.308654
META,0.807452,0.345423,0.869442,1.186909,0.000253,1.221202
AMZN,0.671496,0.363889,0.272153,0.382065,0.000221,1.069872
GOOG,0.796773,0.315238,0.584065,0.802054,0.000228,1.099748
AAPL,0.670223,0.418470,0.102533,0.131340,0.000254,1.228014
NFLX,0.508603,0.481117,0.042452,0.052106,0.000222,1.071395
SP500,1.000000,0.228392,0.373264,0.446150,0.000207,1.000000


In [20]:
# correlation_heatmap = sns.heatmap(correlation, vmin=-1, vmax=1)
# correlation_heatmap

In [33]:
# Make interactive ratios bar plot

ratio_wd = pn.widgets.Select(options= list(ratios_df.columns), name='Ratio')

def ratiosplot_df(ratio_wd):
    df = pd.DataFrame(ratios_df[ratio_wd])
    return df

ratios_bar = hvplot.bind(ratiosplot_df, ratio_wd).interactive(width=600).hvplot.bar(
    title = "Key Ratios",
    xlabel = "Ticker"
    )
ratios_bar

In [22]:
# ratios_df["Volatility"].hvplot.bar(ylabel = "Volatility %")

In [23]:
# ratios_df["Sharpe"].hvplot.bar(ylabel = "Sharpe Ratio")

In [24]:
# ratios_df["Sortino"].hvplot.bar(ylabel = "Sortino Ratio")

In [25]:
# ratios_df["Beta"].hvplot.bar(ylabel = "Betas")

## Panel

In [26]:
#this section is a whiteboard

# tickers = tickrs[0:]

# company_info = yf.Ticker("NIO").info
# company_info
# company_desc = yf.Ticker("NIO").info['longBusinessSummary']
# company_name = yf.Ticker("NIO").info['longName']
# company_website = yf.Ticker("NIO").info['website']
# logourl = yf.Ticker("NIO").info['logo_url']
# type(logourl)

In [40]:
# FIRST TRY

#widgets
# tickers = pn.widgets.Select(name='Ticker', options=[tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr])
df_widget = pn.widgets.DataFrame(ratios_df, name='Key Ratios')

#companyinfo - TBD on how to use widget to automatically update
company_name = yf.Ticker(tickers_wd.value).info['longName']
companydesc = yf.Ticker(tickers_wd.value).info['longBusinessSummary']
companywebsite = yf.Ticker(tickers_wd.value).info['website']
logourl = yf.Ticker(tickers_wd.value).info['logo_url']

#template
template = pn.template.FastListTemplate(
    title = company_name + ' - Stock Analysis',
    sidebar = [
        tickers_wd.servable(area="sidebar"), #HAVING ISSUES TRYING TO USE TICKERS WIDGET TO AUTOMATICALLY UPDATE COMPANY NAME, DESC, AND WEBSITE WHEN USED.
        pn.pane.Markdown("# " + company_name),
        pn.pane.Markdown(companydesc)],
        # pn.bind(tickers,company_name,companydesc,companywebsite)],             HAVING ISSUES TRYING TO USE TICKERS WIDGET TO AUTOMATICALLY UPDATE COMPANY NAME, DESC, AND WEBSITE WHEN USED.
    sidebar_footer = companywebsite,
    main=[
        # tickers_wd,
        pn.Row(pn.Column(prices_plot, margin=(0,25)), 
                 cum_returns_plot), 
        pn.Row(pn.Column(df_widget, margin=(0,25)), 
                 ratios_bar)
               ],
    theme="dark",
    logo = logourl,
)

template.servable();      
template.show()

Launching server at http://localhost:50402


In [28]:
#SECOND TRY 
#Trying to use functions below for interactive Select widget

#widgets
# tickers = pn.widgets.Select(name='Ticker', options=[tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr])
# tickers.servable(area="sidebar")
# df_widget = pn.widgets.DataFrame(ratios_df, name='Key Ratios')


# def company_name(ticker):
#     name = pn.pane.Markdown("# " + yf.Ticker(ticker).info['longName'])
#     return name

# def companydesc(ticker):
#     desc = pn.pane.Markdown(yf.Ticker(ticker).info['longBusinessSummary'])
#     return desc
    
# def companywebsite(ticker):
#     website = pn.pane.Markdown("### " +yf.Ticker(ticker).info['website'])
#     return website

# #template
# template = pn.template.FastListTemplate(
#     title = 'Stock Analysis',
#     sidebar = [
#         tickers, 
#         company_name(tickers.value),
#         companydesc(tickers.value),
#         companywebsite(tickers.value)],
#     main=[
#         pn.Row(pn.Column(prices_plot, margin=(0,25)), 
#                  cum_returns_plot), 
#         pn.Row(pn.Column(df_widget, margin=(0,25)), 
#                  ratios_bar)
#                ],
#     theme="dark"
# )

# template.servable();      
# template.show()

Launching server at http://localhost:50072


In [29]:
# tickers = pn.widgets.Select(name='Ticker', options=[tickr,tickrs[0],tickrs[1],tickrs[2],tickrs[3],tickrs[4],mkttickr])
# df_widget = pn.widgets.DataFrame(ratios_df, name='Key Ratios')


# def company_name(ticker):
#     name = pn.pane.Markdown("# " + yf.Ticker(ticker).info['longName'])
#     return name

# def companydesc(ticker):
#     desc = pn.pane.Markdown(yf.Ticker(tickers.value).info['longBusinessSummary'])
#     return desc
    
# def companywebsite(ticker):
#     website = pn.pane.Markdown("### " +yf.Ticker(tickers.value).info['website'])
#     return website



In [30]:
# title = '## Stock Explorer hvPlot'

# tickers = ['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT']

# def get_df(ticker):
#     df = pd.DataFrame(getattr(stocks, ticker))
#     df['date'] = pd.to_datetime(df.date)
#     return df.set_index('date').mean().reset_index()

# def get_plot(ticker):
#     df = get_df(ticker)
#     return df.hvplot.line('date', 'close', grid=True, responsive=True, height=300)

# ticker = pn.widgets.Select(name='Ticker', options=tickers)


# def get_plot(ticker):
#     df = get_df(ticker)
#     return df.hvplot.line('date', 'close', grid=True, responsive=True, height=300)

# pn.Row(
#     pn.Column(title, ticker),
#     pn.bind(get_plot, ticker)
# )

# ticker.servable(area='sidebar')

# pn.panel("""This example compares **four different implementations of the same app** using 

# - the quick and easy ``interact`` function, 
# - more flexible *reactive* functions,
# - declarative *Param-based* code, and 
# - explicit *callbacks*.""").servable()

# pn.panel(pn.bind(get_plot, ticker, window)).servable(title='hvPlot Stock Explorer');